<a href="https://colab.research.google.com/github/Nithyasri-Babu/customer-segmentation/blob/main/SMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import timedelta

# STEP 1: Load your dataset
df = pd.read_csv('/content/modified_supermarket_data.csv')

# STEP 2: Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='mixed')

# STEP 3: Create a CustomerID (if not already unique)
df['CustomerID'] = df['Customer type'] + '_' + df['Gender'] + '_' + df['Region']

# STEP 4: Calculate Reference Date for Recency
reference_date = df['Date'].max() + timedelta(days=1)

# STEP 5: Calculate Recency, Frequency, and Monetary
recency_df = df.groupby('CustomerID')['Date'].max().reset_index()
recency_df['Recency'] = (reference_date - recency_df['Date']).dt.days

frequency_df = df.groupby('CustomerID')['Invoice ID'].nunique().reset_index()
frequency_df.columns = ['CustomerID', 'Frequency']

monetary_df = df.groupby('CustomerID')['Total'].sum().reset_index()
monetary_df.columns = ['CustomerID', 'Monetary']

# STEP 6: Merge into one RFM DataFrame
rfm = recency_df.merge(frequency_df, on='CustomerID').merge(monetary_df, on='CustomerID')

# STEP 7: Compute quantiles
q_recency = rfm['Recency'].quantile(0.5)
q_frequency = rfm['Frequency'].quantile(0.5)
q_monetary = rfm['Monetary'].quantile(0.5)

# STEP 8: Assign Segments using quantiles
def assign_segment(row):
    if row['Frequency'] > q_frequency and row['Monetary'] > q_monetary:
        return 'Loyal High Spenders'
    elif row['Frequency'] <= q_frequency and row['Recency'] > q_recency:
        return 'At-Risk'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(assign_segment, axis=1)

# OPTIONAL: Merge phone number from original dataset
if 'Phone' in df.columns:
    phone_df = df[['CustomerID', 'Phone']].drop_duplicates()
    rfm = rfm.merge(phone_df, on='CustomerID', how='left')

# Save final result (optional)
rfm.to_csv('/content/rfm_segmented_customers.csv', index=False)

# Display segmented summary
print(rfm['Segment'].value_counts())
rfm.head()


Segment
Others                 573
Loyal High Spenders    350
At-Risk                 77
Name: count, dtype: int64


,CustomerID,Date,Recency,Frequency,Monetary,Segment,Phone
0,Member_Female_Nagapattinam,2019-03-30,1,80,26643.0675,Others,-9952149169
1,Member_Female_Nagapattinam,2019-03-30,1,80,26643.0675,Others,-7695986734
2,Member_Female_Nagapattinam,2019-03-30,1,80,26643.0675,Others,-6637040088
3,Member_Female_Nagapattinam,2019-03-30,1,80,26643.0675,Others,-6047233983
4,Member_Female_Nagapattinam,2019-03-30,1,80,26643.0675,Others,-7776251945


In [ ]:
import pandas as pd

# Load the RFM segmented customer data
rfm = pd.read_csv('/content/rfm_segmented_customers.csv')

# Define message templates for each segment
message_templates = {
    'Loyal High Spenders': "Hi customer, thank you for being a loyal shopper! 🎁 Enjoy 20% OFF on your next visit.",
    'At-Risk': "Hi customer, we miss you! 🧡 Come back and get 15% OFF on your next purchase.",
    'Others': "Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!"
}

# Send messages to each customer (simulated with print)
for index, row in rfm.iterrows():
    customer_id = row['CustomerID']
    phone = row['Phone'] if 'Phone' in rfm.columns else 'Unknown'
    segment = row['Segment'] if pd.notnull(row['Segment']) else 'Others'

    message = message_templates.get(segment, message_templates['Others']).format(customer=customer_id)

    # Simulated SMS
    print(f"[SIMULATED SMS] To: {phone} | Segment: {segment} | Message: {message}")


[SIMULATED SMS] To: -9952149169 | Segment: Others | Message: Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!
[SIMULATED SMS] To: -7695986734 | Segment: Others | Message: Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!
[SIMULATED SMS] To: -6637040088 | Segment: Others | Message: Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!
[SIMULATED SMS] To: -6047233983 | Segment: Others | Message: Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!
[SIMULATED SMS] To: -7776251945 | Segment: Others | Message: Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!
[SIMULATED SMS] To: -7123892776 | Segment: Others | Message: Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!
[SIMULATED SMS] To: -8724856164 | Segment: Others | Message: Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!
[SIMULATED SMS] To: -6744789396 | Segment: Others | Message: Hi customer, 🛍️ explore our latest offers and enjoy 10% OFF!
[SIMULATED SMS] To: -868

In [ ]:
import requests

# Your details
authkey = '460146AwfjiIlBD86874e4c3P1'                  # Replace with your MSG91 auth key
mobile = '919952149260'                         # Your own number (with country code)
message = "Hello! 👋 customer explore our latest offers and enjoy 10% OFF!."

# Use MSG91 test sender ID (no DLT approval needed)
sender = 'MSGIND'                                # Use MSG91 default sender
route = '4'                                      # 4 = transactional
country = '91'

# Compose request URL
url = f"https://api.msg91.com/api/sendhttp.php?mobiles={mobile}&authkey={authkey}&route={route}&country={country}&message={message}&sender={sender}"

# Send request
response = requests.get(url)

# Output result
if response.status_code == 200:
    print("✅ SMS sent successfully to your test number!")
    print("Response:", response.text)
else:
    print("❌ Failed to send SMS")
    print("Status Code:", response.status_code)
    print("Response:", response.text)


✅ SMS sent successfully to your test number!
Response: 35676e72766b4a616d4a7250


In [ ]:
import requests
print(requests.get('https://api.ipify.org').text)


35.245.238.74


In [ ]:
import requests

authkey = '460146AwfjiIlBD86874e4c3P1'
mobile = '919952149260'  # Use your phone number
message = "📲 Test message after IP whitelist."

params = {
    'authkey': authkey,
    'mobiles': mobile,
    'message': message,
    'sender': 'MSGIND',
    'route': '4',
    'country': '91'
}

url = 'https://api.msg91.com/api/sendhttp.php'

response = requests.get(url, params=params)

print("Status:", response.status_code)
print("Response:", response.text)


Status: 200
Response: 35676e72753776696b62454c


In [ ]:
import requests

# Replace these with your actual values
authkey = '460146AwfjiIlBD86874e4c3P1'        # MSG91 Auth Key
sender = 'MSGIND'                     # Registered sender ID (MSG91 provides one for trial)
mobile = '919952149260'               # Your or test number (with country code)
route = '1'                           # 4 = transactional
country = '91'
message = "Hello! 👋 This is a real-time test SMS from MSG91."

# Construct the URL
url = f"https://api.msg91.com/api/sendhttp.php?mobiles={mobile}&authkey={authkey}&route={route}&country={country}&message={message}&sender={sender}"

# Send the GET request
response = requests.get(url)

# Display result
if response.status_code == 200:
    print("✅ SMS sent successfully!")
    print("Response ID:", response.text)
else:
    print("❌ Failed to send SMS")
    print("Status Code:", response.status_code)
    print("Response:", response.text)


✅ SMS sent successfully!
Response ID: 35676e725956634b6b6d4b78


In [ ]:
import requests

# Replace with your actual details
authkey = '460146AwfjiIlBD86874e4c3P1'
mobile = '919952149260'  # Enter your phone number with country code
message = "🎉 Hello! This is a test SMS from our supermarket segmentation project."
sender = 'MSGIND'  # Default test sender
route = '4'  # Transactional
country = '91'

# Construct the URL
url = f"https://api.msg91.com/api/sendhttp.php?mobiles={mobile}&authkey={authkey}&route={route}&country={country}&message={message}&sender={sender}"

# Send request
response = requests.get(url)

# Print result
if response.status_code == 200:
    print("✅ SMS sent successfully!")
    print("Response:", response.text)
else:
    print("❌ Failed to send SMS")
    print("Status Code:", response.status_code)
    print("Response:", response.text)


✅ SMS sent successfully!
Response: 35676e736c49766c31457372


In [ ]:
import requests

authkey = '460146AwfjiIlBD86874e4c3P1'      # 🔁 Replace this
mobile = '919952149260'             # 🔁 Your mobile number
message = "Hello! 👋 This is a test message from MSG91 (no DLT)."
sender = 'MSGIND'                   # Default test sender
route = '4'                         # Transactional route
country = '91'

url = (
    f"https://api.msg91.com/api/sendhttp.php?"
    f"mobiles={mobile}&authkey={authkey}&route={route}"
    f"&country={country}&message={message}&sender={sender}"
)

response = requests.get(url)

if response.status_code == 200:
    print("✅ SMS sent successfully!")
    print("Response:", response.text)
else:
    print("❌ Failed to send SMS")
    print("Status Code:", response.status_code)
    print("Response:", response.text)


✅ SMS sent successfully!
Response: 35676e735036716358425468
